# Implementation of Deep-Walk Tutorial


In this tutorial we are going to learn about the implementation of Deep-Walk Algorithm from screatch.

Following are the Requirements for this Implementation.

1. Python 3.0 +
2. Pytorch
3. DGL (Deep Graph Library)

# Implementation is going to Happen in following phases.

1. Implementation from scratch

2. Using Deep Graph Library

3. Examples of Deep-Walk on Datasets

# Phase-1

From Scratch Implementation using Python:

In [1]:
# Importing nessesary Libraries 

import networkx as nx # to visulaize and load graph data
import random
import numpy as np
from typing import List
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec


# This is Just temporary Code File .. ihavent done any editing yet not added text into that

I will be following function structure and pusing changes soon


# Creating Deep-Walk model class

In [2]:
class DeepWalk:
    def __init__(self, window_size: int, embedding_size: int, walk_length: int, walks_per_node: int):
        """
        :param window_size: window size for the Word2Vec model
        :param embedding_size: size of the final embedding
        :param walk_length: length of the walk
        :param walks_per_node: number of walks per node
        """
        self.window_size = window_size
        self.embedding_size = embedding_size
        self.walk_length = walk_length
        self.walk_per_node = walks_per_node

    def random_walk(self, g: nx.Graph, start: str, use_probabilities: bool = False) -> List[str]:
        """
        Generate a random walk starting on start
        :param g: Graph
        :param start: starting node for the random walk
        :param use_probabilities: if True take into account the weights assigned to each edge to select the next candidate
        :return:
        """
        walk = [start]
        for i in range(self.walk_length):
            neighbours = g.neighbors(walk[i])
            neighs = list(neighbours)
            if use_probabilities:
                probabilities = [g.get_edge_data(walk[i], neig)["weight"] for neig in neighs]
                sum_probabilities = sum(probabilities)
                probabilities = list(map(lambda t: t / sum_probabilities, probabilities))
                p = np.random.choice(neighs, p=probabilities)
            else:
                p = random.choice(neighs)
            walk.append(p)
        return walk

    def get_walks(self, g: nx.Graph, use_probabilities: bool = False) -> List[List[str]]:
        """
        Generate all the random walks
        :param g: Graph
        :param use_probabilities:
        :return:
        """
        random_walks = []
        for _ in range(self.walk_per_node):
            random_nodes = list(g.nodes)
            random.shuffle(random_nodes)
            for node in tqdm(random_nodes):
                random_walks.append(self.random_walk(g=g, start=node, use_probabilities=use_probabilities))
        return random_walks

    def compute_embeddings(self, walks: List[List[str]]):
        """
        Compute the node embeddings for the generated walks
        :param walks: List of walks
        :return:
        """
        model = Word2Vec(sentences=walks, window=self.window_size, vector_size=self.embedding_size)
        return model.wv

In [3]:
G = nx.karate_club_graph()

In [5]:
G.nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33))

In [6]:
G.edges

EdgeView([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 10), (0, 11), (0, 12), (0, 13), (0, 17), (0, 19), (0, 21), (0, 31), (1, 2), (1, 3), (1, 7), (1, 13), (1, 17), (1, 19), (1, 21), (1, 30), (2, 3), (2, 7), (2, 8), (2, 9), (2, 13), (2, 27), (2, 28), (2, 32), (3, 7), (3, 12), (3, 13), (4, 6), (4, 10), (5, 6), (5, 10), (5, 16), (6, 16), (8, 30), (8, 32), (8, 33), (9, 33), (13, 33), (14, 32), (14, 33), (15, 32), (15, 33), (18, 32), (18, 33), (19, 33), (20, 32), (20, 33), (22, 32), (22, 33), (23, 25), (23, 27), (23, 29), (23, 32), (23, 33), (24, 25), (24, 27), (24, 31), (25, 31), (26, 29), (26, 33), (27, 33), (28, 31), (28, 33), (29, 32), (29, 33), (30, 32), (30, 33), (31, 32), (31, 33), (32, 33)])

In [7]:
deep_walk_emb = DeepWalk(5,20,10,2)

In [8]:
walks = deep_walk_emb.get_walks(G)

100%|██████████| 34/34 [00:00<00:00, 17066.34it/s]


In [9]:
emb = deep_walk_emb.compute_embeddings(walks)

In [14]:
len(emb)

34

In [17]:
emb[0]


array([ 0.03112174,  0.0248144 ,  0.04046149,  0.00347368,  0.03297189,
       -0.01910742,  0.0028322 ,  0.03734919, -0.04480325, -0.02202702,
       -0.03131172, -0.00766304,  0.04807733, -0.03980104, -0.00698867,
       -0.01050909,  0.04778976, -0.03033342, -0.0053213 , -0.02617892],
      dtype=float32)

In [18]:
emb[1]

array([-0.01182669,  0.00085064, -0.01528985, -0.03921663, -0.00713159,
        0.01069064,  0.00111983,  0.03442913, -0.01967189,  0.00910139,
        0.03211999,  0.04008023, -0.00738268, -0.04923522,  0.02520437,
        0.00204629,  0.0426208 , -0.00512349, -0.01755557, -0.04554198],
      dtype=float32)